In [1]:
model.obj = Objective(expr=sum(model.Y[i, j] for i in machines for j in products), sense=minimize)

# Define constraints
model.constraints = ConstraintList()

# Each product must pass through machines sequentially
for j in products:
    for i in range(len(machines) - 1):
        model.constraints.add(model.Y[i, j] <= model.X[i+1, j])

# Time constraints for each task on each machine (not provided in the document, placeholders used)
# You need to define these constraints based on the problem description

# Display the decision variables, objective function, and constraints
print("Decision Variables:")
print("Start time (X):", model.X)
print("Finish time (Y):", model.Y)
print("\nObjective Function:")
print(model.obj)
print("\nConstraints:")
print(model.constraints)


In [45]:
# ---------------------------------------------------------------------------- #
#                              Import the modules                              #
# ---------------------------------------------------------------------------- #
import pandas as pd
from pyomo.environ import *

# ---------------------------------------------------------------------------- #
#                             Initialize the MODEL                             #
# ---------------------------------------------------------------------------- #
model = ConcreteModel()

machines = [1, 2, 3, 4, 5]
products = [1, 2, 3]
products_dict = {1:"A",2:"B",3:"C"}

# ---------------------------------------------------------------------------- #
#                                Define the SETS                               #
# ---------------------------------------------------------------------------- #
model.machines = machines
model.products = products

# ---------------------------------------------------------------------------- #
#                             Define the VARIABLES                             #
# ---------------------------------------------------------------------------- #

model.X = Var(model.machines, model.products, within=NonNegativeReals)
model.Y = Var(model.machines, model.products, within=NonNegativeReals)
model.P = Var(within=Binary)
model.F1 = Var(within=Binary)
model.F2 = Var(within=Binary)
model.F3 = Var(within=Binary)
model.G = Var(within=Binary)
model.K = Var(within=Binary)
model.L = Var(within=NonNegativeReals)

# ---------------------------------------------------------------------------- #
#                         Define the OBJECTIVE FUNCTION                        #
# ---------------------------------------------------------------------------- #
model.objective = Objective(expr=model.L, sense=minimize)

# ---------------------------------------------------------------------------- #
#                               Define the PARAMS                              #
# ---------------------------------------------------------------------------- #
times = pd.read_excel('Time Data.xlsx', index_col=0)


model.A = Param(model.machines, initialize=times.loc[:,f'Product {products_dict[1]}'].to_dict().values())
model.B = Param(model.machines, initialize=times.loc[:,f'Product {products_dict[2]}'].to_dict().values())
model.C = Param(model.machines, initialize=times.loc[:,f'Product {products_dict[3]}'].to_dict().values())

# ----------------------------------- Big M ---------------------------------- #
model.M = 1e6

# ---------------------------------------------------------------------------- #
#                            Define the CONSTRAINTS                            #
# ---------------------------------------------------------------------------- #

# -------------------- Linearazation of objective function ------------------- #
model.objective_linear = ConstraintList()
model.objective_linear.add(model.L <= model.Y[4, 1])
model.objective_linear.add(model.L <= model.Y[5, 2])
model.objective_linear.add(model.L <= model.Y[5, 3])

# ---------------------------------- X and Y --------------------------------- #
model.start_and_end_time_relation = ConstraintList()
for machine in machines:
    for product in products:
        model.start_and_end_time_relation.add(model.Y[i, j] >= model.X[i, j])
        model.start_and_end_time_relation.add(model.X[machine, product] + model[product] == model.Y[machine, product])

# ----------- Each product must pass through machines sequentially ----------- #
model.sequential = ConstraintList()
for j in products:
    for i in range(len(machines) - 1):
        model.sequential.add(model.Y[i, j] <= model.X[i+1, j])
model.sequential.add(model.Y[4, 1] <= 60)
model.sequential.add(model.Y[5, 2] <= model[5,3] + 15)


# --------------------- Machines can do something at once -------------------- #

# Precedence constraints based on binary variables
model.precedence_constraints = ConstraintList()
model.precedence_constraints.add(expr=model.P + model.F1 <= 1)
model.precedence_constraints.add(expr=model.P + model.F2 <= 1)
model.precedence_constraints.add(expr=model.P + model.F3 <= 1)

# Solve the model
solver = SolverFactory('clex')
solver.solve(model)



NameError: name 'ConstrantList' is not defined

In [20]:
times = pd.read_excel('Time Data.xlsx', index_col=0)
times.loc[:,"Product A"].to_dict().values()

dict_values([10, 6, 12, 22, 0])

In [46]:
1e6

1000000.0